# Connect Eye-AI and Load Libraries

In [259]:
# repo_dir = "Repos"   # Set this to be where your github repos are located.
# %load_ext autoreload
# %autoreload 2

# # Update the load path so python can find modules for the model
# import sys
# from pathlib import Path
# sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))
# sys.path.insert(0, str(Path.home() / repo_dir / "deriva-ml"))

In [1]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
import numpy as np
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
from pathlib import Path, PurePath
import logging

#from deriva_ml import DatasetBag, Workflow, ExecutionConfiguration
from deriva_ml import DerivaML, Workflow, ExecutionConfiguration, VersionPart
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

2025-03-24 14:34:59.098069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-24 14:34:59.098115: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-24 14:34:59.239103: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 14:34:59.514999: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-24 14:35:01.575672: W tensorflow/compiler/tf2

In [2]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
host = 'www.eye-ai.org'
#host = 'dev.eye-ai.org' #for dev testing
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

2025-03-24 14:35:03,492 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-03-24 14:35:03,494 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-03-24 14:35:03,496 - INFO - Setting up RefreshTokenAuthorizer with auth_client=[instance:139875693761664]
2025-03-24 14:35:03,497 - INFO - Setting up a RenewingAuthorizer. It will use an auth type of Bearer and can handle 401s.
2025-03-24 14:35:03,497 - INFO - RenewingAuthorizer will start by using access_token with hash "5eb628ddd4882be3813093734615056ecacd99e8cc0717c1430965acb42a3e07"
2025-03-24 14:35:03,498 - INFO - Executing token refresh without client credentials
2025-03-24 14:35:03,499 - INFO - Fetching new token from Globus Auth
2025-03-24 14:35:03,938 - INFO - request done (success)
2025-03-24 14:35:03,939 - INFO - RenewingAuthorizer.

You are already logged in.


# Configuration

In [3]:
cache_dir = '/data'
working_dir = '/data'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

2025-03-24 14:35:23,511 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-03-24 14:35:23,512 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>


In [8]:


#ml_instance.increment_dataset_version(dataset_rid='2-N93J', component= VersionPart.patch, description='Update to latest deriva-ml schema')

DatasetVersion(major=2, minor=0, patch=1)

In [4]:
source_dataset = "2-N93J"
asset_RID = ["2-C8JM"]
ml_instance = DerivaML(host, catalog_id="eye-ai")


preds_workflow = EA.add_workflow( 
    Workflow(
        name="LAC data template",
        url="https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/Sandbox_KB/Get_VGGPreds.ipynb",
        workflow_type="Test Workflow",
        )
    )

config = ExecutionConfiguration(
    datasets=[
        {
            "rid": source_dataset,
            "materialize": False,
            "version": ml_instance.dataset_version(source_dataset),
        }
    ],
    assets=asset_RID,
    workflow=preds_workflow,
    description="Instance of linking VGG19 predictions to patient-level data",
    )

exec = ml_instance.create_execution(config)

2025-03-24 14:35:29,800 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-03-24 14:35:29,801 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-03-24 14:35:31,456 - INFO - Loading /home/kb_766/deriva-ml/cache/2-N93J_9a75d3c2b53de08134358a4de4fd06d182218938b6a737bce545ac887954bcb1/Dataset_2-N93J
2025-03-24 14:35:32,882 - INFO - Creating new database for dataset: 2-N93J in /home/kb_766/deriva-ml/cache/2-N93J_9a75d3c2b53de08134358a4de4fd06d182218938b6a737bce545ac887954bcb1/2-N93J@330-1H6W-MEKW.db
2025-03-24 14:35:33,369 - INFO - File [/home/kb_766/deriva-ml/DerivaML_working/4-S42C/asset/predictions_results.csv] transfer successful. 59.57 KB transferred. Elapsed time: 0:00:00.000946.
2025-03-24 14:35:33,369 - INFO - Verifying SHA256 checksum for downloaded file [/home/kb_766/deriva-ml/Der

In [5]:
print(exec)

caching_dir: /home/kb_766/deriva-ml/cache
_working_dir: /home/kb_766/deriva-ml/DerivaML_working
execution_rid: 4-S42C
workflow_rid: 4-M4TT
asset_paths: [PosixPath('/home/kb_766/deriva-ml/DerivaML_working/4-S42C/asset/predictions_results.csv')]
configuration: datasets=[DatasetSpec(rid='2-N93J', materialize=False, version=DatasetVersion(major=2, minor=0, patch=1))] assets=['2-C8JM'] workflow='4-M4TT' description='Instance of linking VGG19 predictions to patient-level data'


# Get Pertinent Datasets

In [99]:
ds_bag = exec.datasets[0]
diagsTall = EA.image_tall(ds_bag, 'Initial Diagnosis')

In [105]:
# Merge datasets to create master dataframe

# Function to update column names
pd.options.mode.copy_on_write = True
def updateCols(df, cols, colDict):
    df = df[cols]
    df.rename( columns = colDict, inplace = True )
    for c in set(cols).intersection( set(colDict) ): cols[cols.index(c)] = colDict.get(c)
    return df

cols = ['Subject_RID', 'Image_RID', 'Image_Side', 'Diagnosis_Image']
colDict = {}
masterDF = updateCols(diagsTall, cols, colDict)

masterDF = pd.merge( masterDF,
        ds_bag.get_table_as_dataframe('Subject'),
        left_on = 'Subject_RID',
        right_on = 'RID',
        how = 'left')

cols.extend(['Subject_ID','Subject_Gender','Subject_Ethnicity'])
colDict = {'Subject_ID':'EyePacs_ID'}
masterDF = updateCols(masterDF, cols, colDict)

masterDF = pd.merge( masterDF,
        ds_bag.get_table_as_dataframe('Image'),
        left_on = 'Image_RID',
        right_on = 'RID',
        suffixes = ('', '_right'),
        how = 'left')

cols.extend(['Observation'])
colDict = {'Observation':'Observation_RID'}
masterDF = updateCols(masterDF, cols, colDict)

masterDF = pd.merge( masterDF,
        ds_bag.get_table_as_dataframe('Observation'),
        left_on = 'Observation_RID',
        right_on = 'RID',
        how = 'left')

cols.extend(['date_of_encounter', 'Age'])
masterDF = updateCols(masterDF, cols, colDict)

patientDF = masterDF.drop_duplicates(subset=['Subject_RID', 'Diagnosis_Image']).reset_index() # patient level
eyeDF = masterDF.drop_duplicates(subset=['Subject_RID', 'Diagnosis_Image', 'Image_Side']).reset_index() # eye level

# Select Test Set

In [106]:
# Target total size ~600 images, so ~300 patients
# Select ~150 case patients

# Drop "Indian subcontinent origin" because there are so few, also none in original test set
caseDF = patientDF[ patientDF['Diagnosis_Image'] == 'Suspected Glaucoma']
caseDF = caseDF[ - caseDF['Subject_Ethnicity'].isin(['Latin American', 'Indian subcontinent origin', '']) ]

# Drop cases if too many, for African, Asian, ethnicity not specified
caseDF = caseDF.drop( caseDF[ caseDF['Subject_Ethnicity'] == 'African Descent'].iloc[0:45].index )
caseDF = caseDF.drop( caseDF[ caseDF['Subject_Ethnicity'] == 'Asian'].iloc[0:22].index )
caseDF = caseDF.drop( caseDF[ caseDF['Subject_Ethnicity'] == 'ethnicity not specified'].iloc[0:81].index )

caseKey, caseCount = np.unique( caseDF['Subject_Ethnicity'], return_counts=True )
caseCounts = dict( zip( caseKey, caseCount ))

In [107]:
# Select controls
cntrlRatio = 1  # meaning 1 case to 2 controls
bins = list(range(0, 101, 10))  # age bins

cntrlDF = patientDF[ patientDF['Diagnosis_Image'] == 'No Glaucoma']
cntrlDF = cntrlDF[ - cntrlDF['Subject_Ethnicity'].isin(['Latin American', 'Indian subcontinent origin', 'Multi-racial', '']) ]

cntrlKey, cntrlCount = np.unique( cntrlDF['Subject_Ethnicity'], return_counts=True )
cntrlCounts = dict( zip( cntrlKey, cntrlCount ))

cntrlRIDs = []
for e in pd.unique( cntrlDF['Subject_Ethnicity'] ):
    needMore = 0
    if cntrlCounts[e] > (cntrlRatio * caseCounts[e]):
        # Enough controls to try gender matching
        for s in pd.unique( caseDF[ caseDF['Subject_Ethnicity'] == e ].loc[:,'Subject_Gender']):
            tempCaseDF = caseDF[ (caseDF['Subject_Ethnicity'] == e) &  (caseDF['Subject_Gender'] == s) ]
            tempCntrlDF = cntrlDF[ (cntrlDF['Subject_Ethnicity'] == e) & (cntrlDF['Subject_Gender'] == s) ]

            if tempCntrlDF.shape[0] > (cntrlRatio * tempCaseDF.shape[0]):
                # Enough to try age matching
                i, c = np.unique( pd.cut(tempCaseDF['Age'], bins = bins), return_counts=True )
                tCounts = dict( zip( i, c ))
                cntrlAges = pd.cut(tempCntrlDF['Age'], bins = bins )
                for ind in i:
                    cntrlMatch = np.where( cntrlAges == ind )[0]
                    if len(cntrlMatch) < ( cntrlRatio * tCounts[ind] ):
                        # Not enough controls for this age bin matching, take all of them
                        cntrlRIDs.extend( tempCntrlDF.loc[ cntrlAges.index[cntrlMatch]].loc[:,'Subject_RID'] )
                        needMore = needMore + ( ( cntrlRatio * tCounts[ind] ) - len(cntrlMatch) )
                    else:
                        # More than enough controls for this age bin matching, only take enough
                        cntrlRIDs.extend( tempCntrlDF.loc[ cntrlAges.index[cntrlMatch]].loc[:,'Subject_RID'].iloc[0:( cntrlRatio * tCounts[ind] )] )
            
                if needMore > 0:
                    # Not enough were age matched, take more gender matched
                    cntrlRIDs.extend( list(set(tempCntrlDF['Subject_RID']) - set(cntrlRIDs))[0:needMore] )
                    needMore = 0

            else:
                # Not enough for gender + age matching in this gender, do ethnicity matching
                # First take all for that gender, then use needMore
                cntrlRIDs.extend(tempCntrlDF['Subject_RID'])
                needMore = needMore + ( (cntrlRatio * tempCaseDF.shape[0]) - tempCntrlDF.shape[0] )

        if needMore > 0:
            # Not enough were gender matched, take more ethnicity matched
            cntrlRIDs.extend( list( set(caseDF[caseDF['Subject_Ethnicity'] == e].loc[:,'Subject_RID']) - set(cntrlRIDs))[0:needMore] )
            needMore = 0
            
    else:
        # Not enough controls for ethnicity + gender + age matching, take all of them
        cntrlRIDs.extend( cntrlDF[ cntrlDF['Subject_Ethnicity'] == e ].loc[:,'Subject_RID'] )

In [117]:
# Put it all together in new TEST SET
testDF = masterDF[ masterDF['Subject_RID'].isin( cntrlRIDs + list( caseDF['Subject_RID'] ) )]
testDF['Subject_RID']

5       2-CDB4
6       2-CDB4
41      2-CDCE
42      2-CDCE
81      2-CDDW
         ...  
6968    2-CN8P
6969    2-CN8P
6985    2-CN9A
6986    2-CN9A
6987    2-CN9A
Name: Subject_RID, Length: 645, dtype: object

# Create Child Dataset to New LAC with New Test Set

In [112]:

test_dataset = exec.create_dataset(['LAC', 'Test'], description='A race/gender/age matched test dataset')
EA.add_dataset_members( dataset_rid = test_dataset, members = testDF['Subject_RID'])

# Upload Results

In [118]:
# # crete asset path
# asset_type_name = "Diagnosis_Analysis"
# asset_path = exec.execution_asset_path(asset_type_name)

# # save assets to asset_path
# linkdDF.to_csv(asset_path/'ImagesToVGG19.csv', index=False)

# upload assets to catalog
exec.upload_execution_outputs(clean_folder=True)

2025-03-24 16:16:09,766 - INFO - Initializing uploader: GenericUploader v1.7.6 [Python 3.10.13, Linux-5.10.210-201.852.amzn2.x86_64-x86_64-with-glibc2.26]
2025-03-24 16:16:09,767 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2025-03-24 16:16:09,767 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2025-03-24 16:16:09,804 - INFO - Scanning files in directory [/home/kb_766/deriva-ml/DerivaML_working/deriva-ml/execution/4-S42C]...
2025-03-24 16:16:09,805 - INFO - Including file: [/home/kb_766/deriva-ml/DerivaML_working/deriva-ml/execution/4-S42C/execution-metadata/Execution_Config/configuration.json].
2025-03-24 16:16:09,806 - INFO - Including file: [/home/kb_766/deriva-ml/DerivaML_working/deriva-ml/execution/4-S42C/execution-metadata/Runtime_Env/environment_snapshot_ytvbrm53.txt].
2025-03-24

{'Execution_Config/configuration.json': FileUploadState(state=<UploadState.success: 0>, status='Complete', result={'URL': '/hatrac/execution_metadata/c574c9dbd4d2f1c98a1120b20c9c2b3a.configuration.json:dzyDIqTmdshSDmojKAIzT8JYnBDH1Mdx', 'RID': '4-S4R0', 'RCT': '2025-03-24T23:16:10.356903+00:00', 'RMT': '2025-03-24T23:16:10.356903+00:00', 'RCB': 'https://auth.globus.org/6022643c-876c-4a47-bafa-5b9fac2c7782', 'RMB': 'https://auth.globus.org/6022643c-876c-4a47-bafa-5b9fac2c7782', 'Filename': 'configuration.json', 'Description': None, 'Length': 226, 'MD5': 'c574c9dbd4d2f1c98a1120b20c9c2b3a', 'Execution_Metadata_Type': 'Execution_Config'}, rid='4-S4R0'),
 'Runtime_Env/environment_snapshot_ytvbrm53.txt': FileUploadState(state=<UploadState.success: 0>, status='Complete', result={'URL': '/hatrac/execution_metadata/4bdd399d7cc638e1bb0e992d330c240b.environment_snapshot_ytvbrm53.txt:yPxwkhgeO6Ub5AcWr6kVDY8hvXhRfPpx', 'RID': '4-S4R2', 'RCT': '2025-03-24T23:16:10.635156+00:00', 'RMT': '2025-03-24T2